In [72]:
import pandas as pd

In [73]:
mat = pd.read_csv("raw_data/student/student-mat.csv", sep=";")
por = pd.read_csv("raw_data/student/student-por.csv", sep=";")

In [74]:
mat['course'] = 'mat'
por['course'] = 'por'

In [75]:
df = pd.concat([mat, por])

In [76]:
df

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3,course
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,3,4,1,1,3,6,5,6,6,mat
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,3,3,1,1,3,4,5,5,6,mat
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,3,2,2,3,3,10,7,8,10,mat
3,GP,F,15,U,GT3,T,4,2,health,services,...,2,2,1,1,5,2,15,14,15,mat
4,GP,F,16,U,GT3,T,3,3,other,other,...,3,2,1,2,5,4,6,10,10,mat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,4,2,1,2,5,4,10,11,10,por
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,3,4,1,1,1,4,15,15,16,por
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,5,6,11,12,9,por
647,MS,M,17,U,LE3,T,3,1,services,services,...,4,5,3,4,2,6,10,10,10,por


In [77]:
df.drop_duplicates(
    subset=[
        "school",
        "sex",
        "age",
        "address",
        "famsize",
        "Pstatus",
        "Medu",
        "Fedu",
        "Mjob",
        "Fjob",
        "reason",
        "nursery",
        "internet",
    ], inplace=True
)

In [78]:
from sklearn.preprocessing import LabelEncoder

def clean_data(df):
    # Multi-label encode column 'school' using delimiter 'GP'
    loc_0 = df.columns.get_loc("school")
    df_encoded = df["school"].str.get_dummies(sep="GP").add_prefix("school_")
    df = pd.concat([df.iloc[:, :loc_0], df_encoded, df.iloc[:, loc_0 + 1 :]], axis=1)
    # Multi-label encode column 'sex' using delimiter 'F'
    loc_0 = df.columns.get_loc("sex")
    df_encoded = df["sex"].str.get_dummies(sep="F").add_prefix("sex_")
    df = pd.concat([df.iloc[:, :loc_0], df_encoded, df.iloc[:, loc_0 + 1 :]], axis=1)
    for col in df.columns:
        if df[col].dtype == "object":
            if len(df[col].unique()) <= 2:
                le = LabelEncoder()
                df[col] = le.fit_transform(df[col])
    df = pd.get_dummies(df)
    return df


df = clean_data(df.copy())
df.head()

,school_MS,sex_M,age,address,famsize,Pstatus,Medu,Fedu,traveltime,studytime,...,Fjob_other,Fjob_services,Fjob_teacher,reason_course,reason_home,reason_other,reason_reputation,guardian_father,guardian_mother,guardian_other
0,0,0,18,1,0,0,4,4,2,2,...,0,0,1,1,0,0,0,0,1,0
1,0,0,17,1,0,1,1,1,1,2,...,1,0,0,1,0,0,0,1,0,0
2,0,0,15,1,1,1,1,1,1,2,...,1,0,0,0,0,1,0,0,1,0
3,0,0,15,1,0,1,4,2,1,3,...,0,1,0,0,1,0,0,0,1,0
4,0,0,16,1,0,1,3,3,1,2,...,1,0,0,0,1,0,0,1,0,0


In [80]:
df['y'] = df['G3'] >= 10

In [81]:
df.to_csv('new_data/student.csv')